# Recomendação de músicas e filmes

## Preparação do ambiente

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode, from_json
from pyspark.sql.types import StringType, ArrayType, IntegerType, StructType, StructField
from re import sub

In [0]:
# Conexão com o Neo4j
neo4j_url = "" # insira a URL da sua instância no neo4j
neo4j_user = "" # insira o usuário da sua instância no neo4j
neo4j_password = "" # insira a senha da sua instância do neo4j
dbname = "" # insira o nome do seu banco de dados do Neo4j
connector_path = "" # insira o caminho do conector específico para a versão para o Apache Spark instalado

In [0]:
spark = (
    SparkSession.builder.config("neo4j.url", neo4j_url)
    .config("neo4j.authentication.basic.username", neo4j_user)
    .config("neo4j.authentication.basic.password", neo4j_password)
    .config("neo4j.database", dbname)
    .config("spark.jars", connector_path)
    .getOrCreate()
)

## Consultas

> 1. Recomendação de músicas baseadas em um filme específico, correlacionando o dataset de filmes e o dataset de trilhas sonoras (pelo nome do filme) com o dataset de músicas, para então recomendar músicas similares através dos atributos de músicas similares do dataset de músicas.

In [0]:
movie_name = "The Wolf of Wall Street"

query = f"""
    // Músicas da trilha sonora
    MATCH (m:Movie {{title: "{movie_name}"}})-[:SOUNDTRACK]->(s:Song)
    RETURN DISTINCT s.song AS Música, s.artist AS Artista
    
    UNION
    
    // Músicas similares às da trilha sonora
    MATCH (m:Movie {{title: "{movie_name}"}})-[:SOUNDTRACK]->(s:Song)
    MATCH (s)-[:SIMILAR]->(recommended:Song)
    RETURN DISTINCT recommended.song AS Música, recommended.artist AS Artista
"""

spark.read.format("org.neo4j.spark.DataSource") \
	.option("query", query) \
	.load() \
	.show(truncate=False)

> 2. Recomendações de músicas similares, utilizando atributos diversos (emoção, gênero, explicidade) do dataset de músicas.

In [0]:
# Consulta de músicas similares por emoção, artistas e músicas similares:
song_id = "pyre"

query = f"""
    MATCH (s:Song {{song_id: "{song_id}"}})-[:DESPERTA]-(e:Emotion)
    MATCH (s2:Song)-[:DESPERTA]-(e)
    WHERE 
    s2.explicit = s.explicit AND s2.song_id <> s.song_id
    RETURN s2.song, s2.explicit, s2.emotion, s2.artist LIMIT 5

    UNION

    MATCH (s:Song {{song_id: "{song_id}"}})-[:INTERPRETADA_POR]-(a:Artist)
    MATCH (s2:Song)-[:INTERPRETADA_POR]-(a)
    WHERE 
    a.artist_name = s2.artist AND s.song_id <> s2.song_id AND s2.explicit = s.explicit
    RETURN s2.song, s2.explicit, s2.emotion, s2.artist LIMIT 5

    UNION

    MATCH (s:Song {{song_id: "{song_id}"}})-[:SIMILAR]-(s2:Song)
    WHERE
    s2.explicit = s.explicit AND s.song_id <> s2.song_id
    RETURN s2.song, s2.explicit, s2.emotion, s2.artist
"""

spark.read.format("org.neo4j.spark.DataSource") \
	.option("query", query) \
	.load() \
	.show(truncate=False)

> 3. Recomendações de filmes similares, utilizando atributos diversos (gênero, linguagem, explicidade) do dataset de filmes.

In [0]:
# Consulta de filmes similares por gênero
identifierByName = "anidealhusband"

query = f"""
    MATCH (m:Movie {{identifierByName: "{identifierByName}"}})-[:HAS_GENRE]-(g:MovieGenre)
    MATCH (m2:Movie)-[:HAS_GENRE]-(g)
    WHERE
        m2.identifierByName <> m.identifierByName
    RETURN m2.title as Filme, g.genre_name as Gênero
"""

spark.read.format("org.neo4j.spark.DataSource") \
	.option("query", query) \
	.load() \
	.show(truncate=False)

# Consultas de filmes similares por diretor:
query = f"""
    MATCH (m:Movie {{identifierByName: "{identifierByName}"}})-[:DIRECTED]-(d:Director)
    MATCH (m2:Movie)-[:DIRECTED]-(d)
    WHERE
        m2.identifierByName <> m.identifierByName
    RETURN m2.title as Filme, d.director_name as Diretor
"""

spark.read.format("org.neo4j.spark.DataSource") \
	.option("query", query) \
	.load() \
	.show(truncate=False)

> 4. Encontrar músicas que tocam em filmes de um diretor específico, usando atributo de diretor do dataset de filmes, atrelado ao datasets de trilhas sonororas.

In [0]:
director_name = "Edward L. Cahn"

query = f"""
	MATCH (d:Director)-[:DIRECTED]-(m:Movie)-[:SOUNDTRACK]-(s:Song)-[:INTERPRETADA_POR]-(a:Artist)
	WHERE d.director_name = "{director_name}"
	RETURN s.song as Música, s.album as Álbum, a.artist_name as Artista, m.title as Filme
"""

spark.read.format("org.neo4j.spark.DataSource") \
	.option("query", query) \
	.load() \
	.show(truncate=False)

> 5. Ligar emoções à filmes específicos, utilizando a combinação do dataset de filmes e trilhas sonoras para atrelar o atributo emoções (do dataset de músicas) à filmes individuais.

In [0]:
movie_name = "The Wolf of Wall Street"

query = f"""
	MATCH (e:Emotion)-[:DESPERTA]-(s:Song)-[:SOUNDTRACK]-(m:Movie)
	WHERE m.title = "{movie_name}"
	RETURN e.emotion as Emoção, s.song as Música
"""

spark.read.format("org.neo4j.spark.DataSource") \
	.option("query", query) \
	.load() \
	.show(truncate=False)